In [22]:
import json

# Initialize an empty array to store the parsed JSON objects
parsed_data = []
routes_done = []

with open('routes_done', 'r') as f:
    for line in f:
        routes_done.append(line.strip()) 

# Open and read the file line by line
try:
    with open('output-new-2025-04-11-0.json', 'r') as file:
        for line in file:
            try:
                # Parse each line as a JSON object and append to the array
                json_obj = json.loads(line.strip())
                parsed_data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON in line: {e}")
                continue
    
    print(f"Successfully parsed {len(parsed_data)} JSON objects from the file.")
except FileNotFoundError:
    print("File 'output-new-2025-04-11-0.json' not found.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
parsed_data

Successfully parsed 1 JSON objects from the file.


[{'860548042828571': {'matched_stops': [0, 22, 23, 24, 25, 26, 27, 28, 30, 32],
   'stops_length': 39,
   'match_percentage': 100.0,
   'trail_segment': [{'lat': 13.033667,
     'timestamp': '2025-04-11 02:02:20',
     'long': 80.2304475,
     'device_id': '860548042828571'},
    {'lat': 13.03315225,
     'timestamp': '2025-04-11 02:02:30',
     'long': 80.23053185,
     'device_id': '860548042828571'},
    {'lat': 13.0323918,
     'timestamp': '2025-04-11 02:02:40',
     'long': 80.2305787,
     'device_id': '860548042828571'},
    {'lat': 13.03180785,
     'timestamp': '2025-04-11 02:02:50',
     'long': 80.23063306666667,
     'device_id': '860548042828571'},
    {'lat': 13.031144716666669,
     'timestamp': '2025-04-11 02:03:00',
     'long': 80.23069045,
     'device_id': '860548042828571'},
    {'lat': 13.030375716666668,
     'timestamp': '2025-04-11 02:03:10',
     'long': 80.23077215,
     'device_id': '860548042828571'},
    {'lat': 13.0301053,
     'timestamp': '2025-04-11 0

In [24]:
# get the best matching polylines data for each route
from geopy.distance import geodesic
routesPolylines={}
closestToStart = {}
for json_data in parsed_data:
    for deviceId in json_data:
        deviceData = json_data[deviceId]
        routeId = deviceData.get('route_id')
        if routeId in routes_done:
            continue
        routeData = deviceData
        routeFirstStop = routeData.get('stops', [])[0]
        routeLastStop = routeData.get('stops', [])[-1]
        trailFirstPoint = routeData.get('trail_segment', [])[0]
        oldList = routeData.get('trail_segment', [])
        routeData['trail_segment'] = [routeFirstStop] + oldList + [routeLastStop]
        distanceFromRouteStart = geodesic((trailFirstPoint['lat'], trailFirstPoint['long']), (routeFirstStop['lat'], routeFirstStop['long'])).meters
        if routeId in routesPolylines:
            olderValue = routesPolylines[routeId]
            if routeData['match_percentage'] == olderValue['match_percentage'] and distanceFromRouteStart < closestToStart[routeId]:
            # if True:
                # routesPolylines[routeId].append(routeData)
                routesPolylines[routeId] = routeData
                closestToStart[routeId] = distanceFromRouteStart
        else:
            # routesPolylines[routeId] = [routeData]
            routesPolylines[routeId] = routeData
            closestToStart[routeId] = distanceFromRouteStart


In [25]:
with open("route_pollines_even_less_fine.json", "w") as f:
    f.write(json.dumps(routesPolylines, indent=2))
    

In [ ]:
# Generate Google polylines for each route's trail segment
import polyline

# Create a hashmap to store the encoded polylines for each route
route_encoded_polylines = {}

for route_id, route_data in routesPolylines.items():
    # Extract trail segment coordinates
    trail_segment = route_data.get('trail_segment', [])
    
    # Format coordinates for polyline encoding (lat, lng format)
    coordinates = []
    for point in trail_segment:
        if 'lat' in point and 'long' in point:
            coordinates.append((point['lat'], point['long']))
    
    # Encode the coordinates as a Google polyline
    if coordinates:
        encoded_polyline = polyline.encode(coordinates)
        route_encoded_polylines[route_id] = encoded_polyline
    else:
        print("failed")

print(f"Generated polylines for {len(route_encoded_polylines)} routes")

# Save the encoded polylines to a file
with open("route_encoded_polylines.json", "w") as f:
    f.write(json.dumps(route_encoded_polylines, indent=2))

print("Encoded polylines saved to route_encoded_polylines.json")


Generated polylines for 1 routes
Encoded polylines saved to route_encoded_polylines.json


In [37]:
# Generate SQL update queries for atlas_app.route table
sql_queries = []

for route_id, encoded_polyline in route_encoded_polylines.items():
    # Create SQL update query
    sql_query = f"UPDATE atlas_app.route SET polyline = '{encoded_polyline}' WHERE code = '{route_id}' and polyline = '';"
    sql_queries.append(sql_query)

# Save the SQL queries to a file
with open("route_polyline_updates.sql", "w") as f:
    for query in sql_queries:
        f.write(query + "\n")


print(f"Generated {len(sql_queries)} SQL update queries")
print("SQL queries saved to route_polyline_updates.sql")


Generated 40 SQL update queries
SQL queries saved to route_polyline_updates.sql


In [ ]:
# Function to decode a polyline string into a list of coordinates
import traceback
def decode_polyline_to_coordinates(encoded_polyline):
    """
    Decode a Google encoded polyline string into a list of (latitude, longitude) coordinates.
    
    Args:
        encoded_polyline (str): The encoded polyline string
        
    Returns:
        list: A list of (latitude, longitude) tuples
    """
    try:
        coordinates = polyline.decode(encoded_polyline)
        return ([{"lat": lat, "lon": lng, "index": i} for i, (lat, lng) in enumerate(coordinates)], coordinates)
    except Exception as e:
        print(f"Error decoding polyline: {e}")
        print(f"Polyline string: {encoded_polyline[:50]}..." if len(encoded_polyline) > 50 else f"Polyline string: {encoded_polyline}")
        print(f"Polyline length: {len(encoded_polyline)}")
        print(f"Exception type: {type(e).__name__}")
        print(f"Stack trace: {traceback.format_exc()}")
        return ([], [])
kepler, coord = decode_polyline_to_coordinates("kspnAw_uhNALfBOvCIrBIdCKvCOt@Kj@GdBOdDKfC?f@}@SkCGaA?cBbCSfCD~@jDf@`CXjBj@zAfAfB|AvBrBfCXVB@d@n@BF`@|@bCvBfC`ClClBhAh@NL`BrApClBvD~@xFyAdEoA~C{@dEmAlBk@NEvA]fBe@lCs@~Cw@hFaBtCs@tAz@gA`DgAnDoAfDoA|DaA~Cw@jC{@dCc@|BLhCh@tBl@|ChAbFx@jCp@zB|A`D`AjCxAnBzBtCbAnABDXNHD??????FBd@R??NJ\\`@NTt@XhBVzA@~Ae@`Dt@vAr@pCx@zCHxBKd@GhA_@n@WlAw@fBoAnAoAvBeBbCuBvAqAhB{AfCyBJIDCTUn@q@lAAdBO`AMlBc@`By@vA_A`BeAdDoAjBcArA_BlByAt@i@r@e@jA_A|@o@jAy@vB{BbBoBlBeCrAgA`@K~AG|AJjBJdCYfBc@lBp@|A|@tCnAPJf@ZnCt@rAf@lAhAx@XlBTfFUnCQ|BCjDA|AGlB@L@????H?`ABj@f@@?d@^rBl@rCbAvFdBpCx@rD`Ad@J??xAf@jCt@zCdAfFzAxDhAbElAzGnBxCx@tC~@n@T\\LdBh@xEvAlDbAjD~@dF`B`D`AbEhAbG`BfD~@pBp@vEnApDjA|EvAfFzAtErA|E~AdFvApFvAbDr@JD??\\NfBp@xB`B|BtCxCzBhAb@@@r@VxBh@lDr@|Dt@vDt@fEv@dDr@zDhAvCBN???p@BrBj@jCp@hD|@lDp@pDXhCLfB`BDfDBjDb@tErBfE~BpBhD|@fDFlD?nD?hETjCZdAXJBb@NvBn@xDz@|DpBjD~CdDvD?pFWlGYjG{@xEkAlBwAjBuAhBiA`@oA~AeCtCc@fCPjCLjBz@rCr@pD`@bCTE??BCIa@?O??????????q@SMi@GkAKkBm@oAw@cAW]ESCoAIyBx@iChBgClB_CdBsBz@oARWj@iA|@_Bv@uCX{DH}CPwBlAs@")
with open("4104-poly.json", "w") as f:
    f.write(json.dumps(coord, indent=2))

with open("4104-poly-kep.json", "w") as f:
    f.write(json.dumps(kepler, indent=2))


In [49]:
import math 
from typing import List

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    using the haversine formula
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Radius of earth in kilometers
    r = 6371
    
    return c * r


def is_point_near_polyline(point_lat, point_lon, polyline_points, max_distance_meter=50):
    """
    Simpler function to check if a point is within max_distance_meter of any 
    segment of the polyline.
    """
    if not polyline_points or len(polyline_points) < 2:
        return False, float('inf'), None
        
    min_distance = float('inf')
    
    min_segment = None
    
    # Check each segment of the polyline
    for i in range(len(polyline_points) - 1):
        # Start and end points of current segment
        p1_lat, p1_lon = polyline_points[i]
        p2_lat, p2_lon = polyline_points[i + 1]
        
        # Calculate distance to this segment using a simple approximation
        # For short segments, this is reasonable and much simpler
        
        # Calculate distances to segment endpoints
        d1 = calculate_distance(point_lat, point_lon, p1_lat, p1_lon)
        d2 = calculate_distance(point_lat, point_lon, p2_lat, p2_lon)
        
        # Calculate length of segment
        segment_length = calculate_distance(p1_lat, p1_lon, p2_lat, p2_lon)
        
        # Use the simplified distance formula (works well for short segments)
        if segment_length > 0:
            # Projection calculation
            # Vector from p1 to p2
            v1x = p2_lon - p1_lon
            v1y = p2_lat - p1_lat
            
            # Vector from p1 to point
            v2x = point_lon - p1_lon
            v2y = point_lat - p1_lat
            
            # Dot product
            dot = v1x * v2x + v1y * v2y
            
            # Squared length of segment
            len_sq = v1x * v1x + v1y * v1y
            
            # Projection parameter (t)
            t = max(0, min(1, dot / len_sq))
            
            # Projected point
            proj_x = p1_lon + t * v1x
            proj_y = p1_lat + t * v1y
            
            # Distance to projection
            distance = calculate_distance(point_lat, point_lon, proj_y, proj_x)
        else:
            # If segment is very short, just use distance to p1
            distance = d1
            
        # Update minimum distance
        if distance < min_distance:
            min_segment = i
            min_distance = distance
            
    # Check if within threshold (convert meters to kilometers)
    max_distance_km = max_distance_meter / 1000
    return min_distance <= max_distance_km, min_distance, min_segment

    # Use the library's implementation when available
def decode_polyline(polyline_str):
    """Wrapper for polyline library's decoder"""
    if not polyline_str:
        return []
    try:
        return polyline.decode(polyline_str)
    except Exception as e:
        print(f"Error decoding polyline: {e}")
        return []


def calculate_route_match_score(vehicle_points: List[dict], max_distance_meter: float = 100) -> float:
    """
    Calculate how well a route matches a series of vehicle_points, considering direction.
    Uses polyline for more accurate route matching when available.
    Returns a score between 0 and 1, where 1 is a perfect match.
    """
    try:
        # Check if stops is a dict with polyline and stops keys
        # route_polyline = "cnqmAuhwgNaCw@aCu@mA_@uMsDeEiA_@|@THzDtArA\lBh@vARd@Dh@N`D~@ZG^RxAb@hAb@~DhA`D|@hJlC`D|@jA^t@RdGdBrGjBNa@aHqBc\gJyDgAoCy@qF_ByS}F{HyBmDcAqIgC}GoBmCiAmW_I}GyBYUaA_@_C}@qBgAk@a@a@UKMm@[eL_IQIKAO?q@e@yF{D_YwRwE_DyB{Aw@o@gBaBs@k@gDyB}E}CqCmBsByAwB{AyAgAaAw@_@_@aAiAi@y@e@}@k@{A_AkCc@{@]e@s@k@g@Yk@SgAWmBUuFQuAKw@OaAy@eBiCa@q@kA_CaCqFkBsEyA_Dq@gB}@iBi@u@sDcFs@cAaByBoBmC}@gAyC_DkAkAsA}AqEgEaBgBIKsBiBqC{C_G}F{J{JmDoDiCqCkCiCsJwJoEoEwFwFkCaC{F}EaFcEaA{@{BiBeFkE}B{Bw@s@e@m@]_@kA{@c@c@iFoFyC}C{CuCiA}@eAy@yB{A_KkHkGyDsCqBoC}BuJaH{B}A}BwA}@e@cCiAoEoBkIqDkGmC}GyCeEcBwB_AsI{DwDgBgCgAiFaCyDcBsCeAaAWq@[kCaAwBs@c@QuBs@sIeDsAk@o@c@o@q@s@aAqC_FKQaHqL[]i@}@w@qAoCwEqBuDuAcCkAsBmAkBaFcIaA}AHIIHu@kA_@_AOk@q@uDk@qC[iAoAeDe@_AcBgDqCiFiIoPgC{EwE}HcAiBMSq@oAuCsES[yDkGaD}EuAeBkBmCgEyFqEmGkAmBc@u@kAyBsBuEmAkCsAmDiBiE}@}BwAkDeAkBmBuBuCoDwDkFw@eAeB}BoBqCyFsHwBwCm@}@k@cAQu@Eu@Bs@RgDRsBDcA@c@Ey@U_Am@aA_@[SM}@_@[KwB]{Cc@MC_B[i@]e@MMGWUq@s@i@cAYu@Yg@e@o@i@o@eAq@qAe@sB_@yACeDLeEBiC?eACmBOcD]u@IwA[aHkBCAeFoAgCg@iCo@iEiAwEeAmAOe@CuDEyFAo@As@GiIeAoHgAq@UuBcAcCmAiAk@}@e@}@WuE{@[GcMgCiAU{@Ov@kKyEe@EGCEoACcBOm@QmF_@gCIeGC{ECyDBeEEiH@yP@sA?iEDgEHcDRaET_@BsBBkBE_BA{@CoAEmAAu@FwARgC^_Ep@{@HwAFiQMcGGwJQwBCiBCsB@yADkBF_AHqAVcAZ}@d@s@^mA|@sCxB{@r@g@h@e@z@o@lAa@z@m@~@uCrEgAtBwCxFi@~@ELHFZVfAsB"
        # polyline_points = decode_polyline(route_polyline)
        polyline_points = [(80.20519,13.068980000000002),(80.20519,13.068980000000002),(80.20519,13.068980000000002),(80.20519,13.068980000000002),(80.20519,13.068980000000002),(80.20519,13.068980000000002),(80.20537,13.06896),(80.20544000000001,13.068600000000002),(80.20545000000001,13.068600000000002),(80.20545000000001,13.068600000000002),(80.20545000000001,13.068600000000002),(80.20545000000001,13.068600000000002),(80.20545000000001,13.068600000000002),(80.20545000000001,13.06858),(80.20541,13.068510000000002),(80.20541,13.06836),(80.20543,13.068240000000001),(80.20542,13.068100000000001),(80.20527000000001,13.06798),(80.20508000000001,13.06803),(80.20498,13.06811),(80.20497,13.068140000000001),(80.20495000000001,13.068190000000001),(80.20495000000001,13.068190000000001),(80.20494000000001,13.068240000000001),(80.20494000000001,13.068270000000002),(80.20493,13.068430000000001),(80.20486000000001,13.068610000000001),(80.20486000000001,13.068620000000001),(80.20486000000001,13.068620000000001),(80.20479,13.06877),(80.20471,13.0691),(80.20476000000001,13.06953),(80.2048,13.06971),(80.20499000000001,13.069880000000001),(80.20537,13.069790000000001),(80.20582,13.06958),(80.20633000000001,13.069230000000001),(80.20697000000001,13.068800000000001),(80.20759000000001,13.0684),(80.20808000000001,13.068100000000001),(80.20833,13.067950000000002),(80.20844000000001,13.06789),(80.20859,13.06775),(80.20891,13.067520000000002),(80.20928,13.067240000000002),(80.20978000000001,13.066870000000002),(80.21017,13.066440000000002),(80.21053,13.06601),(80.21085000000001,13.065620000000001),(80.21112000000001,13.065290000000001),(80.21122000000001,13.065190000000001),(80.21131000000001,13.065090000000001),(80.21142,13.064940000000002),(80.21154000000001,13.0645),(80.21169,13.064000000000002),(80.21169,13.063540000000001),(80.21172,13.062990000000001),(80.21175000000001,13.062420000000001),(80.21166000000001,13.06216),(80.21163,13.061660000000002),(80.21170000000001,13.060950000000002),(80.21165,13.06024),(80.21161000000001,13.059450000000002),(80.2116,13.058650000000002),(80.2116,13.058090000000002),(80.21159,13.057590000000001),(80.21155,13.05713),(80.21152000000001,13.05667),(80.21158000000001,13.056410000000001),(80.21162000000001,13.056130000000001),(80.21159,13.05554),(80.21164,13.05497),(80.21169,13.054760000000002),(80.21169,13.054540000000001),(80.21168,13.054240000000002),(80.21178,13.05386),(80.21180000000001,13.053780000000001),(80.21184000000001,13.05372),(80.21188000000001,13.053350000000002),(80.21198000000001,13.052890000000001),(80.21214,13.05225),(80.21221000000001,13.05142),(80.21216000000001,13.050460000000001),(80.2121,13.049740000000002),(80.21216000000001,13.04897),(80.21223,13.04817),(80.21229000000001,13.047400000000001),(80.21234000000001,13.046740000000002),(80.21238000000001,13.046420000000001),(80.21238000000001,13.046420000000001),(80.21239000000001,13.046310000000002),(80.21238000000001,13.045950000000001),(80.21241,13.045440000000001),(80.21243000000001,13.04521),(80.21242000000001,13.044970000000001),(80.21244,13.04446),(80.21245,13.044010000000002),(80.21243000000001,13.04358),(80.21244,13.04324),(80.21245,13.043090000000001),(80.21245,13.042650000000002),(80.21248000000001,13.04203),(80.21249,13.041360000000001),(80.21254,13.0408),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21257,13.040680000000002),(80.21253,13.04062),(80.21249,13.04025),(80.21249,13.039670000000001),(80.21246000000001,13.03916),(80.21246000000001,13.03894),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21246000000001,13.03893),(80.21245,13.038870000000001),(80.21241,13.038400000000001),(80.2124,13.037970000000001),(80.21243000000001,13.037400000000002),(80.21241,13.037030000000001),(80.2124,13.036470000000001),(80.21241,13.03588),(80.2124,13.035340000000001),(80.21241,13.035330000000002),(80.21241,13.035330000000002),(80.21241,13.035300000000001),(80.21236,13.03494),(80.21224000000001,13.03461),(80.21215000000001,13.034120000000001),(80.21208,13.033800000000001),(80.21204,13.03367),(80.21204,13.03367),(80.21201,13.03344),(80.21196,13.033100000000001),(80.21187,13.03241),(80.21185000000001,13.032090000000002),(80.21173,13.031860000000002),(80.21133,13.031880000000001),(80.21088,13.03198),(80.21041000000001,13.032050000000002),(80.21006000000001,13.032110000000001),(80.20974000000001,13.03208),(80.20956000000001,13.031590000000001),(80.2094,13.031),(80.20928,13.030510000000001),(80.20915000000001,13.03006),(80.20899,13.029560000000002),(80.2089,13.029190000000002),(80.20881,13.02889),(80.20873,13.028620000000002),(80.20856,13.028070000000001),(80.20827000000001,13.02752),(80.20798,13.027070000000002),(80.20763000000001,13.02649),(80.20727000000001,13.02589),(80.20713,13.025530000000002),(80.20703,13.024930000000001),(80.20696000000001,13.02443),(80.20694,13.024270000000001),(80.2069,13.02405),(80.20681,13.02373),(80.20670000000001,13.023380000000001),(80.20675,13.022800000000002),(80.20672,13.02247),(80.20664000000001,13.02213),(80.20657000000001,13.02166),(80.20651000000001,13.021),(80.20652000000001,13.020320000000002),(80.20649,13.019680000000001),(80.20639000000001,13.01918),(80.20635,13.01904),(80.20632,13.018920000000001),(80.20615000000001,13.018510000000001),(80.20605,13.018180000000001),(80.20605,13.018080000000001),(80.20605,13.018080000000001),(80.20605,13.018080000000001),(80.20605,13.018080000000001),(80.20597000000001,13.017930000000002),(80.20585000000001,13.017660000000001),(80.20573,13.017410000000002),(80.20562000000001,13.01716),(80.20539000000001,13.016610000000002),(80.20528,13.016140000000002),(80.20528,13.016090000000002),(80.20527000000001,13.016060000000001),(80.20527000000001,13.016060000000001),(80.20527000000001,13.016060000000001),(80.20528,13.016010000000001),(80.20518000000001,13.01576),(80.20505,13.015450000000001),(80.20498,13.015210000000002),(80.20488,13.01496),(80.20485000000001,13.01486),(80.20485000000001,13.01486),(80.20485000000001,13.01486),(80.20478,13.014690000000002),(80.20468000000001,13.014380000000001),(80.20466,13.014280000000001),(80.20457,13.014000000000001),(80.20451000000001,13.013910000000001),(80.20446000000001,13.01379),(80.20445000000001,13.01378),(80.20445000000001,13.01378),(80.20445000000001,13.01378),(80.20445000000001,13.01378),(80.20445000000001,13.01378),(80.20445000000001,13.01378),(80.20436000000001,13.013530000000001),(80.2042,13.013160000000001),(80.20403,13.0126),(80.20386,13.01181),(80.20376,13.010950000000001),(80.20373000000001,13.010100000000001),(80.20377,13.009250000000002),(80.20384,13.00848),(80.20386,13.00792),(80.20374000000001,13.00726),(80.20337,13.00656),(80.20270000000001,13.00594),(80.20187,13.00552),(80.20141000000001,13.004980000000002),(80.20141000000001,13.00454),(80.20137000000001,13.004240000000001),(80.20127000000001,13.004040000000002),(80.2012,13.003940000000002),(80.20104,13.00364),(80.20092000000001,13.00318),(80.20071,13.002630000000002),(80.2004,13.002120000000001),(80.19978,13.00173),(80.1991,13.00173),(80.1984,13.001840000000001),(80.19775000000001,13.001920000000002),(80.19728,13.00191),(80.19715000000001,13.00191),(80.19715000000001,13.00191),(80.19689000000001,13.001880000000002),(80.19656,13.001740000000002),(80.19622000000001,13.001430000000001),(80.19562,13.00101),(80.19514000000001,13.000620000000001),(80.19470000000001,13.000280000000002),(80.19420000000001,12.999920000000001),(80.19375000000001,12.999640000000001),(80.19326000000001,12.999220000000001),(80.19283,12.998880000000002),(80.19231,12.998450000000002),(80.19162,12.997890000000002),(80.19081000000001,12.99731),(80.19009000000001,12.9967),(80.18965,12.996200000000002),(80.18963000000001,12.99618),(80.18963000000001,12.99618),(80.18963000000001,12.99618),(80.18963000000001,12.99618),(80.18963000000001,12.99618),(80.18963000000001,12.99618),(80.18962,12.996160000000001),(80.18938,12.995940000000001),(80.18895,12.995650000000001),(80.18854,12.995470000000001),(80.18790000000001,12.99515),(80.18725,12.99483),(80.18642000000001,12.994420000000002),(80.18565000000001,12.99402),(80.18502000000001,12.99369),(80.18412000000001,12.99318),(80.18325,12.99266),(80.18248000000001,12.99215),(80.18187,12.991670000000001),(80.18155,12.99144),(80.18153000000001,12.991430000000001),(80.18153000000001,12.991430000000001),(80.1815,12.991420000000002),(80.18138,12.991320000000002),(80.18136000000001,12.99131),(80.18133,12.991290000000001),(80.18124,12.99122),(80.18088,12.990950000000002),(80.18032000000001,12.990490000000001),(80.17982,12.99009),(80.17930000000001,12.989680000000002),(80.17882,12.989300000000002),(80.17837,12.98896),(80.17789,12.98859),(80.17748,12.988270000000002),(80.17707,12.98798),(80.17673,12.987800000000002),(80.17641,12.987570000000002),(80.17599000000001,12.987240000000002),(80.17558000000001,12.986930000000001),(80.17511,12.986590000000001),(80.17464000000001,12.986300000000002),(80.17424000000001,12.98601),(80.17384000000001,12.985740000000002),(80.17334000000001,12.98539),(80.17278,12.985050000000001),(80.17219,12.984670000000001),(80.17156,12.984300000000001),(80.17104,12.98399),(80.17047000000001,12.983630000000002),(80.16974,12.983200000000002),(80.16889,12.98272),(80.16810000000001,12.982230000000001),(80.16742,12.981810000000001),(80.16681000000001,12.981440000000001),(80.16625,12.981140000000002),(80.16566,12.980830000000001),(80.16503,12.980540000000001),(80.16445,12.980290000000002),(80.16383,12.9801),(80.16311,12.979880000000001),(80.16253,12.979640000000002),(80.16248,12.97962),(80.16237000000001,12.979560000000001),(80.16193000000001,12.979280000000001),(80.16127,12.97882),(80.16051,12.978290000000001),(80.15976,12.97774),(80.15901000000001,12.977240000000002),(80.15817000000001,12.97671),(80.15740000000001,12.976210000000002),(80.15675,12.975750000000001),(80.15615000000001,12.975320000000002),(80.15556000000001,12.97488),(80.15490000000001,12.974430000000002),(80.15424,12.973970000000001),(80.15352,12.97352),(80.15289000000001,12.97314),(80.15232,12.97273),(80.15181000000001,12.972370000000002),(80.15133,12.971860000000001),(80.15094,12.971190000000002),(80.15064000000001,12.97056),(80.15037000000001,12.970040000000001),(80.15029000000001,12.969710000000001),(80.15016,12.969270000000002),(80.15015000000001,12.9692),(80.15,12.96897),(80.14975000000001,12.96846),(80.14947000000001,12.967960000000001),(80.14928,12.96752),(80.14919,12.967310000000001),(80.14903000000001,12.966990000000001),(80.14883,12.96654),(80.14853000000001,12.965980000000002),(80.14822000000001,12.965390000000001),(80.14784,12.96479),(80.14754,12.964300000000001),(80.14724000000001,12.96363),(80.1469,12.962980000000002),(80.14652000000001,12.962330000000001),(80.14623,12.961760000000002),(80.14607000000001,12.96141),(80.14598000000001,12.961290000000002),(80.14589000000001,12.96114),(80.14572000000001,12.960830000000001),(80.14537,12.960170000000002),(80.14495000000001,12.9594),(80.14453,12.958630000000001),(80.14425,12.95809),(80.14395,12.957460000000001),(80.14381,12.956980000000001),(80.14362000000001,12.956660000000001),(80.14361000000001,12.956660000000001),(80.14358,12.956640000000002),(80.14342,12.95644),(80.14327,12.956190000000001),(80.14308000000001,12.95584),(80.14279,12.95532),(80.14244000000001,12.954690000000001),(80.1422,12.954250000000002),(80.14187000000001,12.95363),(80.14161,12.953130000000002),(80.14121,12.952560000000002),(80.14088000000001,12.952140000000002),(80.1405,12.951670000000002),(80.14030000000001,12.95132),(80.14026000000001,12.951250000000002),(80.14026000000001,12.951250000000002),(80.14016000000001,12.951110000000002),(80.13992,12.950830000000002),(80.13979,12.95068),(80.13977000000001,12.95067),(80.13976000000001,12.950650000000001),(80.13958000000001,12.95039),(80.13913000000001,12.94987),(80.13860000000001,12.94926),(80.13803,12.94852),(80.1375,12.94777),(80.13716000000001,12.94725),(80.13713000000001,12.9472),(80.13691,12.94691),(80.13647,12.94636),(80.13597,12.945680000000001),(80.13543,12.945010000000002),(80.13497000000001,12.944400000000002),(80.13453000000001,12.943850000000001),(80.1341,12.94339),(80.1336,12.942870000000001),(80.13308,12.942340000000002),(80.13253,12.941820000000002),(80.13199,12.94133),(80.13139000000001,12.940760000000001),(80.13080000000001,12.940190000000001),(80.13031000000001,12.939670000000001),(80.12982000000001,12.93913),(80.12932,12.93856),(80.12887,12.938030000000001),(80.12848000000001,12.93753),(80.12826000000001,12.937230000000001),(80.12806,12.936990000000002),(80.12795000000001,12.936850000000002),(80.12795000000001,12.936850000000002),(80.12778,12.936700000000002),(80.12757,12.936520000000002),(80.12736000000001,12.93627),(80.12713000000001,12.936020000000001),(80.12676,12.935630000000002),(80.12627,12.935120000000001),(80.12575000000001,12.934550000000002),(80.12521000000001,12.933950000000001),(80.1246,12.933340000000001),(80.12394,12.9327),(80.12344,12.932150000000002),(80.12322,12.931870000000002),(80.12320000000001,12.93186),(80.12311000000001,12.931790000000001),(80.12270000000001,12.931410000000001),(80.1222,12.930950000000001),(80.12171000000001,12.930460000000002),(80.12113000000001,12.929870000000001),(80.12049,12.92923),(80.11975000000001,12.92852),(80.11901,12.927800000000001),(80.1183,12.927150000000001),(80.11765000000001,12.926440000000001),(80.11731,12.92604),(80.11731,12.925770000000002),(80.11709,12.925550000000001),(80.11683000000001,12.925260000000002),(80.11676000000001,12.925180000000001),(80.11676000000001,12.925180000000001),(80.11675000000001,12.925170000000001),(80.11674000000001,12.925170000000001),(80.1165,12.92509),(80.11633,12.925110000000002),(80.11596,12.924740000000002),(80.11543,12.924220000000002),(80.11492000000001,12.923710000000002),(80.11448,12.923300000000001),(80.114,12.92279),(80.11349000000001,12.92229),(80.11294000000001,12.92172),(80.11236000000001,12.92116),(80.11189,12.920700000000002),(80.11131,12.92013),(80.11067000000001,12.919580000000002),(80.10992,12.919020000000002),(80.10922000000001,12.918420000000001),(80.10848,12.917840000000002),(80.10784000000001,12.917330000000002),(80.10721000000001,12.916950000000002),(80.10682000000001,12.916730000000001),(80.10677000000001,12.9167),(80.10637000000001,12.916490000000001),(80.10575,12.916160000000001),(80.10502000000001,12.915750000000001),(80.10425000000001,12.915350000000002),(80.10341000000001,12.914900000000001),(80.10266,12.914470000000001),(80.10195,12.913900000000002),(80.10171000000001,12.913170000000001),(80.10167000000001,12.912410000000001),(80.10158000000001,12.911660000000001),(80.10142,12.91106),(80.10105,12.910380000000002),(80.10047,12.909880000000001),(80.09969000000001,12.909500000000001),(80.09917,12.90924),(80.09906000000001,12.90919),(80.09901,12.909160000000002),(80.09892,12.90909),(80.09866000000001,12.90891),(80.09830000000001,12.908650000000002),(80.09790000000001,12.9082),(80.09747,12.907630000000001),(80.09703,12.90708),(80.09660000000001,12.90652),(80.09625000000001,12.906080000000001),(80.09611000000001,12.9059),(80.09609,12.90586),(80.09598000000001,12.90567),(80.09576000000001,12.905360000000002),(80.09549000000001,12.904980000000002),(80.09517000000001,12.904510000000002),(80.09423000000001,12.90333),(80.0938,12.902790000000001),(80.09334000000001,12.902270000000001),(80.09291,12.90174),(80.09247,12.901180000000002),(80.09205,12.900580000000001),(80.0917,12.900120000000001),(80.09127000000001,12.899600000000001),(80.0908,12.89897),(80.09033000000001,12.898380000000001),(80.08994000000001,12.897870000000001),(80.08964,12.897480000000002),(80.08933,12.897060000000002),(80.08894000000001,12.89657),(80.08850000000001,12.895950000000001),(80.08799,12.8953),(80.08749,12.89464),(80.087,12.89395),(80.08648000000001,12.89328),(80.08600000000001,12.89263),(80.08571,12.892190000000001),(80.08561,12.89202),(80.08534,12.891560000000002),(80.08504,12.890910000000002),(80.08477,12.890300000000002),(80.08448000000001,12.889590000000002),(80.08412000000001,12.88873),(80.08378,12.88789),(80.08344000000001,12.887010000000002),(80.08305,12.88606),(80.08269,12.88517),(80.08233000000001,12.884360000000001),(80.08212,12.883590000000002),(80.0818,12.88283),(80.08148000000001,12.88208),(80.08127,12.88151),(80.08098000000001,12.880820000000002),(80.08079000000001,12.88028),(80.08077,12.880220000000001),(80.08073,12.88011),(80.08069,12.880030000000001),(80.08069,12.880030000000001),(80.08063000000001,12.879880000000002),(80.08049000000001,12.879650000000002),(80.08029,12.87916),(80.08001,12.878430000000002),(80.07973000000001,12.877720000000002),(80.07949,12.87715),(80.0792,12.876310000000002),(80.07889,12.875470000000002),(80.07877,12.875150000000001),(80.07866000000001,12.8749),(80.07845,12.87438),(80.07838000000001,12.87409),(80.07838000000001,12.87409),(80.07838000000001,12.87409),(80.07838000000001,12.87409),(80.07838000000001,12.87409),(80.07838000000001,12.87409),(80.07837,12.874350000000002)]
        min_points_required = 4
        if not vehicle_points or len(vehicle_points) < min_points_required:
            print("here")
            return 0.0

        # Sort vehicle_points by timestamp to ensure they're in chronological order
        vehicle_points = sorted(vehicle_points, key=lambda x: x.get('timestamp', 0))
    
        if polyline_points:
            # Count how many vehicle_points are near the polyline
            near_points = []
            total_distance = 0.0
            
            max_distance_km = max_distance_meter / 1000
            min_segments = set()
            for point in vehicle_points:
                try:
                    is_near, distance, min_segment_start = is_point_near_polyline(
                        point['lat'], point['lon'], polyline_points, max_distance_meter
                    )
                    if is_near:
                        if min_segment_start is not None:
                            min_segments.add(min_segment_start)
                        near_points.append(point)
                        total_distance += distance
                except (KeyError, ValueError, TypeError) as e:
                    print(f"Error checking if point is near polyline: {e}, point: {point}")
                    continue
            
            # Calculate proximity score (0-1)
            proximity_ratio = len(near_points) / len(vehicle_points) if len(vehicle_points) > 0 else 0
            
            # Only proceed if enough vehicle_points are near the polyline
            if proximity_ratio >= 0.3:
                # Convert set to list and sort to check direction
                min_segments_list = list(min_segments)
                if len(min_segments_list) >= 2 and min_segments_list[0] < min_segments_list[-1]:
                    print(f"Route Id: {len(near_points)}/{len(vehicle_points)}, Score: {proximity_ratio:.2f}")
                    return proximity_ratio
            return 0.0
    except Exception as e:
        print(f"Error calculating route match score: {e}\nTraceback")
        return 0.0
    


print(calculate_route_match_score([{"lat": 13.019038016666666, "lon": 80.206296, "timestamp": 1745694352}, {"lat": 13.018816016666667, "lon": 80.20623245, "timestamp": 1745694372}, {"lat": 13.018297033333333, "lon": 80.20601888333333, "timestamp": 1745694382}, {"lat": 13.017492466666667, "lon": 80.20566283333334, "timestamp": 1745694392}, {"lat": 13.016512616666667, "lon": 80.20528975, "timestamp": 1745694402}, {"lat": 13.015839433333333, "lon": 80.20520546666667, "timestamp": 1745694412}, {"lat": 13.0153443, "lon": 80.20497353333333, "timestamp": 1745694422}, {"lat": 13.014516566666666, "lon": 80.20467328333334, "timestamp": 1745694432}, {"lat": 13.013308533333333, "lon": 80.20418521666667, "timestamp": 1745694442}, {"lat": 13.012200566666667, "lon": 80.20394955, "timestamp": 1745694452}, {"lat": 13.010988566666667, "lon": 80.2037698, "timestamp": 1745694462}, {"lat": 13.009454183333334, "lon": 80.20371506666666, "timestamp": 1745694472}, {"lat": 13.008380766666667, "lon": 80.20379911666667, "timestamp": 1745694482}, {"lat": 13.007308766666666, "lon": 80.2037589, "timestamp": 1745694492}, {"lat": 13.0060118, "lon": 80.20279845, "timestamp": 1745694502}, {"lat": 13.005355983333333, "lon": 80.20173586666667, "timestamp": 1745694512}, {"lat": 13.004235483333334, "lon": 80.20129543333333, "timestamp": 1745694522}, {"lat": 13.003532866666667, "lon": 80.20107785, "timestamp": 1745694532}, {"lat": 13.002821966666666, "lon": 80.20085335, "timestamp": 1745694542}, {"lat": 13.002070533333333, "lon": 80.20035081666667, "timestamp": 1745694552}, {"lat": 13.001700716666667, "lon": 80.19936575, "timestamp": 1745694562}, {"lat": 13.001847916666666, "lon": 80.19831306666667, "timestamp": 1745694572}, {"lat": 13.001921133333333, "lon": 80.19693851666666, "timestamp": 1745694582}, {"lat": 13.00135925, "lon": 80.19602723333334, "timestamp": 1745694592}, {"lat": 13.000730116666666, "lon": 80.19517015, "timestamp": 1745694602}]))



0.0


In [10]:
import pandas as pd

# try:
#   mp.set_start_method('spawn', force=True)
# except RuntimeError:
#   # Method already set
  # pass

# Assuming the files are in the current working directory.
# If not, provide the full path to the files.
try:
  # Import necessary libraries for PostgreSQL connection
  import psycopg2
  from psycopg2 import sql
  import io
  
  date = '2025-04-11'
  # PostgreSQL connection parameters - you'll need to replace these with your actual credentials
  pg_params = {
          'dbname': 'mtc_master_prod',
                  'user': 'mtc_root_user',
                          'password': 'C@uM7a$2025',
                                  'host': '10.6.156.30',
                                          'port': '5432'

  }
  
  # try:
  #     # Connect to PostgreSQL
  #     conn = psycopg2.connect(**pg_params)
  #     cursor = conn.cursor()
  #     # Execute the query to get vehicle_route_mapping data
  #     query = f"""
  #     SELECT
  #       "public"."waybills"."waybill_id" AS "waybill_id",
  #       "public"."waybills"."vehicle_no" AS "vehicle_no",
  #       "public"."waybills"."schedule_trip_id" AS "schedule_trip_id",
  #       "Bus Schedule Trip Detail - Schedule Trip"."route_number_id" AS "route_id"
  #     FROM
  #       "public"."waybills"
  #       INNER JOIN "public"."bus_schedule_trip_detail" AS "Bus Schedule Trip Detail - Schedule Trip" 
  #       ON "public"."waybills"."schedule_trip_id" = "Bus Schedule Trip Detail - Schedule Trip"."schedule_trip_id"
  #     WHERE
  #       ("public"."waybills"."duty_date" = '{date}')
  #       AND ("public"."waybills"."deleted" = FALSE)
  #     ORDER BY
  #       "public"."waybills"."vehicle_no" DESC
  #     """
      
  #     # Create a StringIO object to store the CSV data
  #     csv_data = io.StringIO()
      
  #     # Execute the query and fetch the results
  #     cursor.execute(query)
      
  #     # Write header to the StringIO object
  #     header = ['waybill_id', 'vehicle_no', 'schedule_trip_id', 'route_id']
  #     csv_data.write(','.join(header) + '\n')
      
  #     # Write data rows to the StringIO object
  #     for row in cursor.fetchall():
  #         csv_data.write(','.join(str(item) for item in row) + '\n')
      
  #     # Reset the position to the beginning of the StringIO object
  #     csv_data.seek(0)
      
  #     # Close the cursor and connection
  #     cursor.close()
  #     conn.close()
      
  #     # Read the CSV data into a pandas DataFrame
  #     vehicle_route_mapping = pd.read_csv(csv_data)
      
  # except (Exception, psycopg2.Error) as error:
  #     print(f"Error connecting to PostgreSQL database: {error}")
  #     # If there's an error with the PostgreSQL connection, fall back to reading from CSV file
  #     print("Falling back to reading from CSV file...")
  vehicle_route_mapping = pd.read_csv("vehicle_route_mapping-11-april.csv")
  route_stop_mapping = pd.read_csv("route-stop-mapping.csv")
  vehicle_device_mapping = pd.read_csv("vehicle_device_mapping.csv")

  print("vehicle_route_mapping:")
  print(vehicle_route_mapping.head())  # Print the first few rows for verification

  print("\nroute_stop_mapping:")
  print(route_stop_mapping.head())

  print("\nvehicle_device_mapping:")
  print(vehicle_device_mapping.head())

except FileNotFoundError:
  print("One or more of the specified files were not found. Please ensure the files exist in the correct directory.")
except pd.errors.ParserError:
  print("Error parsing the CSV file. Please check the file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

print(vehicle_device_mapping.shape, route_stop_mapping.shape, vehicle_route_mapping.shape)

vehicle_route_mapping.dropna(inplace=True)
route_stop_mapping.dropna(subset=["Stop ID"], inplace=True)
vehicle_device_mapping.dropna(inplace=True)

print(vehicle_device_mapping.shape, route_stop_mapping.shape, vehicle_route_mapping.shape)

# prompt: # prompt: create a dictionary using these 3 files for mapping a device_id to a dictionary of route_id having corresponding stop lat longs

# Assuming the files are in the current working directory.
# If not, provide the full path to the files.

location_csv_file = date + "-location.csv"

query = f"""
            SELECT
                `atlas_kafka`.`amnex_direct_data`.`lat` AS `lat`,
                `atlas_kafka`.`amnex_direct_data`.`timestamp` AS `timestamp`,
                `atlas_kafka`.`amnex_direct_data`.`long` AS `long`,
                `atlas_kafka`.`amnex_direct_data`.`deviceId` AS `device_id`
            FROM `atlas_kafka`.`amnex_direct_data`
            WHERE
                (`atlas_kafka`.`amnex_direct_data`.`timestamp` >= parseDateTimeBestEffort('{date} 00:00:00.000'))
                AND (`atlas_kafka`.`amnex_direct_data`.`timestamp` < parseDateTimeBestEffort('{date} 23:59:59.999'))
            ORDER BY `device_id`, `timestamp`
            """

# prompt: given a clickhouse_client, create pd dataframe with the query above
import os
if os.path.exists(location_csv_file):
    print(f"Loading data from existing file: {location_csv_file}")
    df = pd.read_csv(location_csv_file)
    # The CSV likely has an index column added by pandas when it was saved
    # Only keep the actual data columns we need
    if len(df.columns) == 5:  # If there are 5 columns (includes index)
        df = df.iloc[:, 1:5]  # Take columns 1-4 (skipping the first one which is likely the index)
    # Ensure column names are set correctly
    df.columns = ['lat', 'timestamp', 'long', 'device_id']
    df['device_id'] = df['device_id'].astype(str)
else:
    import clickhouse_driver

    clickhouse_conn_params = {
        
            'host': '10.6.155.15',
                          'port': '9000',
                                        'user': 'juspay_rw',
                                                      'password': '6phlcAo88qfpZrwH',
                                                                    'database': 'atlas_kafka'

    }
    clickhouse_client = clickhouse_driver.Client(
        host=clickhouse_conn_params['host'],
        port=clickhouse_conn_params['port'],
        user=clickhouse_conn_params['user'],
        password=clickhouse_conn_params['password'],
        database=clickhouse_conn_params['database'],
        connect_timeout=clickhouse_conn_params.get('connect_timeout', 10),
        send_receive_timeout=clickhouse_conn_params.get('send_receive_timeout', 30),
        sync_request_timeout=clickhouse_conn_params.get('sync_request_timeout', 30)
    )
    print(f"File {location_csv_file} not found. Querying database...")

    df = pd.DataFrame(clickhouse_client.execute(query))
    df.columns = ['lat', 'timestamp', 'long', 'device_id']
    print(df.head()) 
    df.to_csv(location_csv_file, index=False)  # Save without index column
print("====== locationShape: ", df.shape)

# prompt: convert the timestamp field in df to time strings
df['timestamp'] = pd.to_datetime(df['timestamp'])

# # Convert the datetime objects to strings in the desired format
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(df.head())
d = df.groupby('device_id')
# Call the function to process the data at the end of the if __name__ == "__main__" block
try:
  all_devices_live_data = list(set(df['device_id'].astype(str)))
  route_id_device_details_mapping = {}
  count = 0
  for index, row in vehicle_device_mapping.iterrows():
    device_id = str(int(row['device_id']))
    vehicle_id = row['vehicle_no']
    if device_id not in all_devices_live_data:
      print(f"Device ID {device_id} not found in location data, skipping...")
      count += 1
      continue

    route_ids = vehicle_route_mapping[vehicle_route_mapping['vehicle_no'] == vehicle_id]['route_id']

    for route_id in route_ids:
      if pd.notna(route_id):  # Check for NaN values in route_id
        # Sort stops by sequence
        route_stops = route_stop_mapping[route_stop_mapping['Route ID'] == route_id].sort_values(by='Sequence')
        stop_lat_longs = []
        for index2, row2 in route_stops.iterrows():
          stop_lat = row2['LAT']
          stop_long = row2['LON']
          seq = row2['Sequence']
          stop_lat_longs.append({'lat': stop_lat, 'long': stop_long, 'seq': seq})

        if route_id not in route_id_device_details_mapping:
            route_id_device_details_mapping[route_id] = {}
        route_id_device_details_mapping[route_id][device_id] = stop_lat_longs
        
except FileNotFoundError:
  print("One or more of the specified files were not found. Please ensure the files exist in the correct directory.")
except pd.errors.ParserError:
  print("Error parsing the CSV file. Please check the file format.")
except KeyError as e:
  print(f"A required column is missing: {e}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

print("devices not found: ", count)
import time
time.sleep(5)





vehicle_route_mapping:
   waybill_id vehicle_no  schedule_trip_id  route_id
0     1298560    TNK0765               624      1024
1     1295751    TNK0765               624      1024
2     1295751    TNK0765               624      1017
3     1295751    TNK0765               624      1010
4     1295751    TNK0765               624       965

route_stop_mapping:
   Route ID ROUTE   Stop ID  Sequence                        Name        LAT  \
0         3   102  lsFGVtlq         1          ADYAR BUS TERMINUS  13.006657   
1         3   102  UFoaQvGg         2     BSNL TELEPHONE EXCHANGE  13.000734   
2         3   102  RfEbENxd         3                 ADYAR DEPOT  12.998488   
3         3   102  jQaLNViL         4  THIRUVANMIYUR RATION KADAI  12.988915   
4         3   102  WynPMslR         5                 S.R.P TOOLS  12.978614   

         LON     SOURCE             DESTIN DIRECTION  STAGEID  STAGENO  \
0  80.253205  ADYAR B.T  SIRUSERI I.T PARK        UP        3        7   
1  80.256

In [18]:
import pandas as pd
from multiprocessing import Pool, cpu_count
import multiprocessing as mp
# Make sure date is defined globally


from geopy.distance import geodesic
import numpy as np
import json


def match_stops(route_id, location_trail, stops, matched_routes, matched_routes_intermediate, finding_intermediate=False, threshold_distance=0.08, start_point_distance_threshold_km=0.02):
  first_stop = stops[0]
  potential_starts = []
  last_potential_start = -10

  for i, trail_point in enumerate(location_trail):
    distance = geodesic(
        (trail_point['lat'], trail_point['long']),
        (first_stop['lat'], first_stop['long'])
    ).kilometers
    if distance < start_point_distance_threshold_km:
      if i - last_potential_start < 5:
        potential_starts = potential_starts[:-1]
      last_potential_start = i
      potential_starts.append(i)

  # For each potential starting point, try to match the entire route
  best_match = None
  best_score = float('inf')  # Lower is better
  for start_idx in potential_starts:
      current_trail_idx = start_idx
      matched_stops = [0]  # Start with first stop matched

      last_stop_idx = None
      # Try to match subsequent stops
      for stop_idx in range(1, len(stops)):
          current_stop = stops[stop_idx]
          current_stop_seq = current_stop['seq']
          best_distance = float('inf')
          best_trail_idx = None
          dist_sum = 0
          # latLongs = []
          # Search forward in the trail for the best match for this stop
          for trail_idx in range(current_trail_idx, len(location_trail)):
              trail_point = location_trail[trail_idx]
              distance = geodesic(
                  (trail_point['lat'], trail_point['long']),
                  (current_stop['lat'], current_stop['long'])
              ).kilometers
              if trail_idx - 1 >= 0:
                trail_point_prev = location_trail[trail_idx - 1]
                trail_dist = geodesic(
                    (trail_point['lat'], trail_point['long']),
                    (trail_point_prev['lat'], trail_point_prev['long'])
                ).kilometers
                # latLongs.append({'lat': trail_point['lat'], 'long': trail_point['long']})
                dist_sum += trail_dist
              if distance < threshold_distance:
                  best_distance = distance
                  best_trail_idx = trail_idx
                  break
          # If we found a reasonable match
          # print("dist_sum: ", dist_sum, json.dumps(latLongs), best_trail_idx)
          if dist_sum < 3 and best_distance < threshold_distance:
              matched_stops.append(current_stop_seq)
              last_stop_idx = stop_idx
              current_trail_idx = best_trail_idx
          else:
              # Stop not found, break the search
              break
      # Calculate match score (percentage of stops matched and average distance)
      if len(matched_stops) > 1:  # At least two stops must match
          end_idx = current_trail_idx
          match_percentage = len(matched_stops) / len(stops)
          if match_percentage < best_score and match_percentage >= 1.0:
              segment_trail = location_trail[start_idx:end_idx+1]
              if route_id in matched_routes_intermediate:
                  print("found from here", route_id)
                  oldRouteState = matched_routes_intermediate[route_id]
                  oldRouteState['trail_segment'] = oldRouteState['trail_segment'] + segment_trail
                  oldRouteState['stops'] = oldRouteState['stops'] + stops
                  stops = oldRouteState['stops']
                  segment_trail = oldRouteState['trail_segment']
              if route_id in matched_routes_intermediate:
                  del matched_routes_intermediate[route_id]
              best_score = match_percentage
              best_match = {
                  'matched_stops': matched_stops,
                  'stops_length': len(stops),
                  'match_percentage': match_percentage * 100,
                  'trail_segment': segment_trail,
                  'stops': stops,
                  'start_index': start_idx,
                  'end_index': end_idx,
                  'route_id': route_id
              }
          elif len(matched_stops) > 3:
              updated_last_matching_stop = {}
              updated_last_matching_stop['lat'] = location_trail[end_idx + 1]['lat']
              updated_last_matching_stop['long'] = location_trail[end_idx + 1]['long']
              updated_last_matching_stop['seq'] = stops[last_stop_idx]['seq']
              this_stop_to_rest_missing_stops = [updated_last_matching_stop] + stops[last_stop_idx+1:]
              if route_id not in matched_routes_intermediate:
                matched_routes_intermediate[route_id] = {
                  'matched_stops': matched_stops,
                  'missing_sequences': this_stop_to_rest_missing_stops,
                  'match_percentage': match_percentage * 100,
                  'trail_segment': location_trail[start_idx:end_idx+1],
                  'stops': stops,
                }
                print("missing 2", route_id, json.dumps(matched_routes_intermediate[route_id]))
              else:
                oldRouteState = matched_routes_intermediate[route_id]
                old_matched_stops = oldRouteState['matched_stops']
                if old_matched_stops[0] == matched_stops[0] and len(matched_stops) > len(old_matched_stops):
                  oldRouteState['matched_stops'] = matched_stops
                  oldRouteState['missing_sequences'] = this_stop_to_rest_missing_stops
                  oldRouteState['trail_segment'] = location_trail[start_idx:end_idx+1]
                  oldRouteState['stops'] = stops
                  matched_routes_intermediate[route_id] = oldRouteState
                  print("missing 3", route_id, json.dumps(matched_routes_intermediate[route_id])) 
                elif matched_stops[0] >= old_matched_stops[-1]:
                  oldRouteState['matched_stops'] = oldRouteState['matched_stops'] + matched_stops
                  oldRouteState['missing_sequences'] = this_stop_to_rest_missing_stops
                  oldRouteState['trail_segment'] = oldRouteState['trail_segment'] + location_trail[start_idx:end_idx+1]
                  oldRouteState['stops'] = oldRouteState['stops'] + stops
                  matched_routes_intermediate[route_id] = oldRouteState
                  print("missing 4", route_id, json.dumps(matched_routes_intermediate[route_id]))
  if best_match and not finding_intermediate:
    matched_routes[route_id] = best_match


In [ ]:
# -*- coding: utf-8 -*-
"""Untitled3.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1vIPQUeiavxP-7zvxNTXLODpfhvE9ipTn
"""

# prompt: write code to read json files vehicle_route_mapping.csv, route-stop-mapping.csv, vehicle_device_mapping.csv

import pandas as pd
from multiprocessing import Pool, cpu_count
import multiprocessing as mp
# Make sure date is defined globally


from geopy.distance import geodesic
import numpy as np
import json


def match_trail_to_routes(route_id, location_trail, stops, processed_routes, matched_routes, matched_routes_intermediate):
    # For each route, try to find a matching segment in the trail
    if len(stops) < 2 or route_id in processed_routes:  # Skip routes with fewer than 2 stops
        return
    print("route_id:",route_id)
    if route_id not in matched_routes:
      match_stops(route_id, location_trail, stops, matched_routes, matched_routes_intermediate)

import pandas as pd
from geopy.distance import geodesic
import numpy as np
import json
import os
from tqdm import tqdm

# ... (Your existing code for data loading and preprocessing) ...

output_file="output-new-"
processed_devices_file="processed_devices-new-"
processed_routes_file="routes_done"

# Function to process a single device
def process_chunk(args):
    chunk_id, d, route_id, device_details, output_file, processed_devices_file, processed_routes, date = args
    print("chunk_id", chunk_id)
    matched_routes = {}
    matched_routes_intermediate = {}
    for dId in device_details:
      stops = device_details[dId]
      try:
        # Filter the dataframe for the current device_id and convert to list of dictionaries
        device_data = d.get_group(dId).to_dict('records')
        match_trail_to_routes(route_id, device_data, stops, processed_routes, matched_routes, matched_routes_intermediate)
        with open(processed_devices_file + date + str(chunk_id), 'a') as f:
          f.write(f"{dId}\n")
      except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        print(f"Detailed error traceback for device {dId}:")
        print(error_details)
        print(f"Error processing device {dId} in chunk {chunk_id}: {e}")
    beforeLoop = 0
    while route_id in matched_routes_intermediate:
      oldMatchedRoutesIntermediate = matched_routes_intermediate[route_id]
      afterLoop = len(oldMatchedRoutesIntermediate['stops'])
      print("beforeLoop", beforeLoop)
      print("afterLoop", afterLoop)
      if afterLoop > beforeLoop:
        beforeLoop = len(oldMatchedRoutesIntermediate['stops'])
        for dId in device_details:
          stopsList = oldMatchedRoutesIntermediate['missing_sequences']
          device_data = d.get_group(dId).to_dict('records')
          match_trail_to_routes(route_id, device_data, stopsList, processed_routes, matched_routes, matched_routes_intermediate)
      else:
         break
    res = matched_routes.get(route_id)
    if res:
      with open(output_file + date + '-' + str(chunk_id) + '.json', 'a') as f:
        json.dump({dId: res}, f)
        f.write('\n')
    return f"Chunk {chunk_id} completed"

processed_devices = set()
if os.path.exists(processed_devices_file):
    with open(processed_devices_file, 'r') as f:
        for line in f:
            processed_devices.add(line.strip())

processed_routes = set()

if os.path.exists(processed_routes_file):
    with open(processed_routes_file, 'r') as f:
        for line in f:
            processed_routes.add(line.strip())

devices_to_process = []
for i, (dId, group) in enumerate(d):
  if dId not in processed_devices:
    # Convert DataFrame group to list of dictionaries for better serialization
    devices_to_process.append((dId, group.to_dict('records')))


# Prepare arguments for multiprocessing
# Determine optimal number of processes based on CPU count
num_threads = 1
num_processes = min(num_threads, cpu_count())
print(f"Using {num_processes} processes for parallel processing")

# Create output files for each process
for i in range(num_processes):
    open(output_file + date + str(i), 'w').close()
    open(processed_devices_file + date + str(i), 'w').close()

# Distribute devices evenly across processes - manually split the list
# Calculate chunk size for even distribution
total_devices = len(devices_to_process)
chunk_size = total_devices // num_processes
remainder = total_devices % num_processes


done_routes = ['1010', '1013', '1017', '1030', '1034', '1035', '1093', '1299', '136', '1389', '1440', '1447', '1458', '1486', '1490', '1556', '1569', '1572', '1592', '1620', '1676', '1711', '1735', '1750', '1771', '1772', '1779', '178', '18', '1814', '182', '1824', '1846', '1849', '1858', '1873', '1887', '1913', '1921', '1935', '1936', '1939', '1941', '1946', '1948', '1956', '1961', '1967', '1988', '1993', '200', '2016', '2044', '2046', '2047', '2048', '2050', '2051', '2053', '2055', '2056', '2058', '2061', '2064', '2067', '2068', '2100', '2101', '2104', '2113', '2117', '2119', '2120', '2122', '2125', '2126', '2139', '2146', '2147', '2149', '2154', '2155', '2156', '2158', '2159', '2168', '2169', '2187', '2189', '2194', '2196', '2199', '22', '2201', '2206', '2221', '2223', '2225', '2226', '2229', '2231', '2233', '2235', '2238', '2240', '2241', '2245', '2247', '2249', '2251', '2253', '2255', '2263', '2265', '2270', '2271', '2297', '2298', '2302', '2304', '2315', '2322', '2325', '2326', '2331', '2337', '2347', '2353', '2355', '2356', '2363', '2384', '2385', '2389', '2390', '2409', '2410', '2411', '2413', '2425', '2441', '2442', '2448', '2466', '2470', '2471', '2479', '2487', '2488', '2507', '2513', '2514', '2518', '2520', '2594', '2614', '2615', '2637', '2638', '2676', '2703', '2707', '2709', '2716', '2722', '2730', '2731', '2744', '2746', '2758', '2764', '2765', '2768', '2769', '2776', '2793', '28', '2801', '2814', '2818', '2825', '2826', '2834', '2835', '2842', '2862', '2863', '2864', '2874', '2902', '2903', '2904', '2906', '2909', '2910', '2911', '2912', '2913', '2914', '2921', '2930', '2932', '2948', '2949', '2968', '2969', '2970', '2992', '2993', '30', '3000', '3004', '3005', '3008', '3014', '3016', '3018', '3020', '3021', '3038', '3039', '3063', '3079', '3088', '3089', '3106', '3117', '3121', '3122', '3127', '3128', '3133', '3160', '3161', '3172', '3173', '3177', '319', '3204', '3213', '3214', '3221', '3222', '3226', '3229', '3230', '3239', '3240', '3248', '325', '326', '3284', '3285', '3323', '3327', '333', '3358', '337', '3384', '3385', '3397', '3398', '3399', '3401', '3407', '3408', '3409', '3412', '3413', '3417', '3418', '3419', '3428', '3432', '3434', '3438', '344', '3446', '3448', '3458', '3459', '346', '3465', '3480', '3510', '3514', '3539', '354', '3541', '3542', '3554', '3562', '3590', '3592', '3593', '3594', '3595', '3598', '3599', '3602', '3603', '3604', '3605', '3606', '3616', '3640', '3641', '3658', '3661', '3662', '3683', '3684', '3722', '3723', '3724', '3725', '3728', '3729', '3734', '3735', '3742', '3743', '3746', '3748', '3750', '3755', '3758', '3759', '3762', '3763', '3770', '3771', '3791', '3792', '3799', '38', '3800', '3805', '384', '385', '3855', '3856', '3857', '386', '3861', '3863', '3865', '3866', '3867', '3868', '3873', '3876', '3877', '3878', '3881', '3882', '3887', '3888', '3893', '3913', '3914', '3923', '3924', '3949', '3950', '3975', '3976', '3986', '3990', '3999', '40', '4001', '4006', '4008', '4012', '4017', '4021', '4025', '4027', '4036', '4056', '4060', '4062', '4080', '4082', '4084', '4089', '4090', '4092', '4094', '4096', '410', '4100', '4102', '4103', '4105', '4106', '4108', '4109', '4130', '4132', '4133', '4144', '4145', '4150', '4156', '4157', '4162', '4163', '4171', '4174', '4185', '4199', '420', '4222', '4235', '4236', '4237', '4238', '4240', '4243', '4244', '4247', '4253', '4278', '4285', '4315', '4316', '4320', '4325', '4348', '4355', '4356', '437', '4377', '4378', '44', '4412', '4413', '4419', '4429', '4430', '4435', '4436', '4437', '4438', '4439', '4444', '4463', '4464', '4465', '4466', '4473', '4475', '4477', '4479', '4480', '4496', '4497', '4498', '4503', '4506', '4508', '4510', '4515', '4517', '4519', '452', '4522', '4523', '4524', '4525', '4547', '4549', '4551', '4559', '4561', '4562', '4563', '4564', '4565', '4566', '4567', '4568', '4569', '4570', '4571', '4579', '4586', '4587', '4590', '4591', '4594', '4596', '4597', '4599', '4600', '4601', '4602', '4610', '4611', '4615', '4617', '4622', '4624', '4627', '4628', '4629', '4632', '4633', '4654', '4655', '4659', '4660', '4667', '4692', '4693', '4721', '4722', '4724', '4730', '4731', '4736', '4778', '4783', '4802', '4803', '4851', '4852', '4853', '4860', '4867', '4868', '4874', '4876', '4880', '4933', '4979', '4981', '4986', '4988', '5017', '5018', '5035', '5107', '5109', '5117', '5172', '5176', '5183', '5194', '5195', '522', '527', '542', '555', '565', '608', '650', '692', '695', '699', '701', '702', '759', '789', '8', '808', '841', '844', '845', '862', '866', '936', '942', '944', '98']
# with open("routes_done", "r") as f:
#    done_routes = f.readlines()
#    done_routes = list(map(lambda x: x.replace("\n", ""), done_routes))
# Add a loading indicator to show progress
from tqdm import tqdm
i = 0
# Create a list of arguments for parallel processing
process_args = []
print("route_id already done", done_routes)
for route_id in route_id_device_details_mapping:
    if str(route_id) not in done_routes and route_id == 1002:
      device_details = route_id_device_details_mapping[route_id]
      process_args.append((i, d, route_id, device_details, output_file, processed_devices_file, processed_routes, date))
      i += 1
    else:
        print("route_id already done 2", route_id)

# Use multiprocessing to process routes in parallel
total_routes = len(process_args)
print(f"Processing {total_routes} routes in parallel...")

chunks = []
for i in range(num_processes):
  chunks.append(process_args[i::num_processes])


# Main processing function

    # with Pool(processes=num_processes) as pool:
    #     for _ in tqdm(
    #         pool.imap_unordered(process_chunk, process_args),
    #         total=len(process_args),
    #         desc="Processing routes",
    #         unit="route",
    #         ncols=100,
    #         bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{percentage:3.0f}%]'
    #     ):
    #         pass

Using 1 processes for parallel processing
route_id already done ['1010', '1013', '1017', '1030', '1034', '1035', '1093', '1299', '136', '1389', '1440', '1447', '1458', '1486', '1490', '1556', '1569', '1572', '1592', '1620', '1676', '1711', '1735', '1750', '1771', '1772', '1779', '178', '18', '1814', '182', '1824', '1846', '1849', '1858', '1873', '1887', '1913', '1921', '1935', '1936', '1939', '1941', '1946', '1948', '1956', '1961', '1967', '1988', '1993', '200', '2016', '2044', '2046', '2047', '2048', '2050', '2051', '2053', '2055', '2056', '2058', '2061', '2064', '2067', '2068', '2100', '2101', '2104', '2113', '2117', '2119', '2120', '2122', '2125', '2126', '2139', '2146', '2147', '2149', '2154', '2155', '2156', '2158', '2159', '2168', '2169', '2187', '2189', '2194', '2196', '2199', '22', '2201', '2206', '2221', '2223', '2225', '2226', '2229', '2231', '2233', '2235', '2238', '2240', '2241', '2245', '2247', '2249', '2251', '2253', '2255', '2263', '2265', '2270', '2271', '2297', '2298',

In [19]:
for i in range(len(process_args)):
  process_chunk(process_args[i])

chunk_id 0
route_id: 1002
route_id: 1002
missing 2 1002 {"matched_stops": [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12], "missing_sequences": [{"lat": 12.9975467, "long": 80.2162308, "seq": 12}, {"lat": 12.99078392, "long": 80.22023578, "seq": 13}, {"lat": 12.99128834, "long": 80.21952299, "seq": 14}, {"lat": 12.98770469, "long": 80.22308184, "seq": 15}, {"lat": 12.98540264, "long": 80.22330789, "seq": 16}, {"lat": 12.97617132, "long": 80.22125344, "seq": 17}, {"lat": 12.97107294, "long": 80.21910222, "seq": 18}, {"lat": 12.96362273, "long": 80.21538161, "seq": 19}, {"lat": 12.96301795, "long": 80.21592979, "seq": 20}, {"lat": 12.9539308, "long": 80.21226293, "seq": 21}, {"lat": 12.94862372, "long": 80.21027844, "seq": 22}, {"lat": 12.94405014, "long": 80.20834064, "seq": 23}, {"lat": 12.93968088, "long": 80.20558242, "seq": 24}, {"lat": 12.93398709, "long": 80.20398907, "seq": 25}, {"lat": 12.92944822, "long": 80.20279665, "seq": 26}, {"lat": 12.92632849, "long": 80.19760757, "seq": 27}, {"lat"